## STRUCTS

sizeof() se puede utilizar para obtener el tamaño de bytes que ocupa en memoria una clase, struct o variable.

In [8]:
struct Ejemplo {
    int num;
    char c;
};

printf("MAIN\n=======\n");
Ejemplo ej;
printf("%lu", sizeof(Ejemplo));

return 0;


MAIN
8

Estamos en un caso en el que el compilador ha añadido padding, espacio vacío extra para que los datos de nuestro struct estén alineados a la palabra del procesador. El procesador es mucho más rápido accediendo a memoria alineada. 

Cada tipo tiene un padding asociado a él, esto se traduce a “el espacio de memoria mínimo que tiene que haber entre dos elementos del mismo tipo”.

El orden de elementos hace que structs con los mismos datos, ocupen más. Es muy importante siempre al crear structs grandes, ordenar los elementos de mayor a menor(o viceversa) para evitar espacio extra por el padding.

Podemos sobreescribir el padding:

In [1]:
//pragma pack(1) //VISUAL STUDIO
struct Ejemplo {
    int num;
    char c;
};
// __attribute__((packed)) //CLANG y GCC

printf("MAIN\n=======\n");
Ejemplo ej;
printf("%lu", sizeof(Ejemplo));

return 0;


MAIN
8

## UNIONS

Los elementos de una Union no estan seguidos, sino que ocupan la misma memoria.

Utilidad:
- Polimorfismo de objetos sin crear indirecciones o funciones virtuales.
- Reutilización de memoria sin hacer reinterpr_cast o similares.

Solo puede estar *un solo* elemento activo de un union al mismo tiempo, en C++, acceder a un elemento de la union que no este activo se considera undefined behavior!!

Las unions y los structs pueden combinarse perfectamente. Podemos declarar una struct sin nombre dentro del union
para indicar que esos elementos van a estar en secuencia y al revés.

## CLASES

Una clase, a efectos teóricos, es un struct cuyos elementos pueden
tener private, protected, public para poder acceder a ellos y puede contener métodos.

Sin embargo, en C++ un struct una clase es exactamente lo mismo, la única diferencia es la visibilidad por defecto. En class es private y en struct public.

Las clases (y structs) de C++ tienen la particularidad de que sus
métodos pueden ser const.

Un método const es un método que no puede modificar ninguno de los
datos de la instancia:

In [10]:
class MyClass 
{
    public:
    int GetScore() const
    {
        a += 1;
       return a;    
    }
    
    private:
        int a;
};

MyClass obj;
int score = obj.GetScore();

input_line_16:6:11: error: cannot assign to non-static data member within const member function 'GetScore'
        a += 1;
        ~ ^
input_line_16:4:9: note: member function 'MyClass::GetScore' is declared const here
    int GetScore() const
    ~~~~^~~~~~~~~~~~~~~~


Interpreter Error: 

Si tenemos una instancia const de una clase. Solamente podremos usar métodos que son const ya que el objeto no podemos modificarlo.

In [9]:
class MyClass 
{
    public:
    MyClass() 
    {
        score = 0;
    }
    void AddScore() 
    {
        score += 1;
    }
    
    int GetScore() const
    {
       return score;    
    }
    
    private:
        int score;
};

const MyClass obj;
int score = obj.GetScore();
obj.AddScore();

input_line_15:24:1: error: 'this' argument to member function 'AddScore' has type 'const __cling_N510::MyClass', but function is not marked const
obj.AddScore();
^~~
input_line_15:8:10: note: 'AddScore' declared here
    void AddScore() 
         ^


Interpreter Error: 

Disponemos de una inicialización que ocurre antes del cuerpo del constructor que se llama member initializer. (Solo de esta manera podemos inicializar variables constantes)

In [1]:
class MyClass 
{
    public:
    MyClass()
    {
        score = 0;
        other = 0;
    }
    
    private:
        int score;
        const int other;
};

input_line_7:4:5: error: constructor for 'MyClass' must explicitly initialize the const member 'other'
    MyClass()
    ^
input_line_7:12:19: note: declared here
        const int other;
                  ^
input_line_7:7:15: error: cannot assign to non-static data member 'other' with const-qualified type 'const int'
        other = 0;
        ~~~~~ ^
input_line_7:12:19: note: non-static data member 'other' declared const here
        const int other;
        ~~~~~~~~~~^~~~~


Interpreter Error: 

In [3]:
class MyClass 
{
    public:
    MyClass() : score(0), other(0)
    {
    }
    
    private:
        int score;
        const int other;
};

También podemos asignar el valor inicial por defecto en la propia
variable. Lo cual es aún más legible:

In [ ]:
class MyClass 
{
    public:
    MyClass()
    {
        score = 0;
        other = 0;
    }
    
    private:
    int score;
    const int other;
};

## HERENCIA

Si no ponemos nada, se asumirá la visibilidad por defecto. La cual es private si el padre es una clase, y public si es un struct. Los member initializer también deben de llamar a los constructores de clases padre, proveyendo los argumentos de construcción:

In [28]:
class VirtualClass 
{
    public:
    VirtualClass(int initialScore) : score(initialScore)
    {
    }
    
    virtual void func() 
    {
        printf("padre");
    }
    
    private:
    int score;
};

class VirtualSubClass : public VirtualClass 
{
    public:
    VirtualSubClass() : VirtualClass(0) //score = 0
    {
    } 
    
    virtual void func() override 
    {
        printf("hijo\n");
    }
};

VirtualSubClass subclass;
subclass.func();

hijo

## POLIMORFISMO

In [29]:
//Puntero
VirtualSubClass subclass;
VirtualClass* ptr = &subclass;
ptr->func();

//Alias
VirtualClass& pRef = subclass;
pRef.func();

hijohijo

En el momento que abstraemos Hijo como Padre, C++ no sabe realmente qué objeto estamos tratando de acceder mediante func(), imprimirá “Padre”, o “Hijo” ?
Para solucionar este problema, lo que ocurre por debajo es que se crean punteros a funciones dentro de cada instancia del objeto para saber a qué objeto saltar.
La implementación final, es una lista de punteros a funciones para cada función virtual que exista. Como se puede apreciar, tener esta lista de punteros tiene un coste, en el tamaño y en el tiempo de ejecución, ya que se necesita de un salto extra a la hora de llamar a la función. Esta lista de funciones se conoce comúnmente como VTABLE.

El atributo “final” se puede utilizar para indicar que una funcion o clase no van a poder ser sobrecargados:

In [5]:
class VirtualClass 
{
    public:
    VirtualClass(int initialScore) : score(initialScore)
    {
    }
    
    virtual void func() 
    {
        printf("padre");
    }
    
    private:
    int score;
};

//class VirtualSubClass2 final : public VirtualClass  //clase final
class VirtualSubClass2 : public VirtualClass 
{
    public:
    VirtualSubClass2() : VirtualClass(0)
    {
    } 
    
    virtual void func() override final //método final
    {
        printf("hijo\n");
    }
};

## DESTRUCTORES